In [6]:
import pandas as pd
import os, datetime
import psycopg2
from sqlalchemy import create_engine

# Lista de arquivos

In [10]:
data_folder = '../dados/anp_combustivel/'

list_csvs = [os.path.join(data_folder, file) for file in os.listdir(data_folder) if file.endswith('.csv')]

print(list_csvs)

['../dados/anp_combustivel/ca-2018-01.csv', '../dados/anp_combustivel/ca-2018-02.csv', '../dados/anp_combustivel/ca-2019-01.csv', '../dados/anp_combustivel/ca-2019-02.csv', '../dados/anp_combustivel/ca-2020-01.csv', '../dados/anp_combustivel/ca-2020-02.csv', '../dados/anp_combustivel/ca-2021-01.csv', '../dados/anp_combustivel/ca-2021-02.csv', '../dados/anp_combustivel/ca-2022-01.csv', '../dados/anp_combustivel/ca-2022-02.csv']


# Nomes de colunas

In [11]:
new_cols = ['regiao','uf','municipio','revenda','cnpj','nome_rua','numero_rua','complemento',
'bairro','cep','produto','data_coleta','valor_venda','valor_compra','unidade_medida','bandeira']

# Concatenação de csvs e conversão de tipos de dados

In [12]:
final_df_anp = pd.DataFrame()

for file in list_csvs:
    print(file)

    try:
        df_anp = pd.read_csv(file, sep=';', encoding='utf-8')
    except:
        df_anp = pd.read_csv(file, sep=';', encoding='latin-1')


    df_anp[['Valor de Venda','Valor de Compra']] = df_anp[['Valor de Venda','Valor de Compra']].astype(str).apply(lambda x: x.str.replace(',','.'))
    df_anp[['Valor de Venda','Valor de Compra']] = df_anp[['Valor de Venda','Valor de Compra']].astype(float)

    df_anp['Data da Coleta'] = pd.to_datetime(df_anp['Data da Coleta'], format='%d/%m/%Y')

    if len(final_df_anp) == 0:
        final_df_anp = df_anp.copy()
    else:
        final_df_anp = pd.concat([final_df_anp, df_anp], axis=0)

final_df_anp.columns = new_cols

../dados/anp_combustivel/ca-2018-01.csv
../dados/anp_combustivel/ca-2018-02.csv
../dados/anp_combustivel/ca-2019-01.csv
../dados/anp_combustivel/ca-2019-02.csv
../dados/anp_combustivel/ca-2020-01.csv
../dados/anp_combustivel/ca-2020-02.csv


C:\Users\Jackson Simionato\AppData\Local\Temp\ipykernel_24696\2545707987.py:7: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  df_anp = pd.read_csv(file, sep=';', encoding='utf-8')


../dados/anp_combustivel/ca-2021-01.csv
../dados/anp_combustivel/ca-2021-02.csv
../dados/anp_combustivel/ca-2022-01.csv
../dados/anp_combustivel/ca-2022-02.csv


# Seleção de colunas de interesse

In [13]:
final_df_anp_db = final_df_anp.drop(columns=['valor_compra'])
final_df_anp_db.head()

,regiao,uf,municipio,revenda,cnpj,nome_rua,numero_rua,complemento,bairro,cep,produto,data_coleta,valor_venda,unidade_medida,bandeira
0,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,NaN,STIEP,41770-395,GNV,2018-01-02,2.370,R$ / m³,PETROBRAS DISTRIBUIDORA S.A.
1,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,NaN,STIEP,41770-395,DIESEL S10,2018-01-02,3.240,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
2,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,NaN,STIEP,41770-395,ETANOL,2018-01-02,2.930,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
3,NE,BA,SALVADOR,PETROBRAS DISTRIBUIDORA S.A.,34.274.233/0015-08,RUA EDISTIO PONDE,474,NaN,STIEP,41770-395,GASOLINA,2018-01-02,3.620,R$ / litro,PETROBRAS DISTRIBUIDORA S.A.
4,S,RS,CANOAS,METROPOLITANO COMERCIO DE COMBUSTIVEIS LTDA,88.587.589/0001-17,AVENIDA GUILHERME SCHELL,6340,NaN,CENTRO,92310-000,GNV,2018-01-02,2.699,R$ / m³,BRANCA


In [14]:
final_df_anp_db.to_csv('../dados/anp_combustivel/anp_combustivel_2018_2022.csv', index=False, encoding='utf-8')

# Conexão com banco

In [15]:
user = 'postgres'
password = 'postgres'
host = 'localhost:5432'
database = 'ANP'
table_name = 'preco_combustivel'
schema_name = 'anp'

conn_string = f'postgresql+psycopg2://{user}:{password}@{host}/{database}'


In [16]:
engine = create_engine(conn_string)

In [20]:
connction = engine.connect()

In [22]:
connction.execute('SELECT version()')

ObjectNotExecutableError: Not an executable object: 'SELECT version()'

In [23]:
from sqlalchemy.exc import OperationalError
from sqlalchemy import text
try:
    # Test the connection by making a dummy query (e.g., selecting version for PostgreSQL)
    with engine.connect() as connection:
        result = connection.execute(text('SELECT version()'))
        print("Connection successful")
        for row in result:
            print("Database version:", row[0])
except OperationalError as e:
    print("Connection failed:", e)

Connection successful
Database version: PostgreSQL 16.0, compiled by Visual C++ build 1935, 64-bit


In [27]:
final_df_anp_db.to_sql(name=table_name, con=engine, schema=schema_name, if_exists='replace', index=False, method='multi')